# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.  

Ответьте на вопрос и обоснуйте решение:   

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
1. Изменится. Приведите примеры матриц.
2. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.  

Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.  

Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.  

***Описание данных***

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
Целевой признак: количество страховых выплат клиенту за последние 5 лет.


## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import sys

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,LinearRegression

In [2]:
try:
    df = pd.read_csv('/datasets/insurance.csv')
except FileNotFoundError:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
def df_info(df):
    print(80*"+")
    df.info()
    print(80*"+")
    print('Описание количественных переменных:')
    print(df.describe(percentiles=[.25,.5]))
    print(80*"+")
    print(f'количество дубликатов в таблице = {df.duplicated().sum()}')
    
df_info(df)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Описание количественных переменных:
               Пол      Возраст      Зарплата  Члены семьи  Страховые выплаты
count  5000.000000  5000.000000   5000.000000  5000.000000        5000.000000
mean      0.499000    30.952800  39916.360000     1.194200           0.148000
std       0.500049     8.440807   9900.083569     1.091387           0.463183
min       0.000000    18.000000   5300.000000     0.000000           0.000000
25%       0.000000    24.000000  33300.000000

Видим 153 дубликата. Избавимся от них.

In [4]:
df.loc[df.duplicated(keep=False)].sort_values(by='Зарплата').head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2955,1,32.0,21600.0,0,0
2988,1,32.0,21600.0,0,0
361,0,50.0,24700.0,1,2
2869,0,50.0,24700.0,1,2
333,0,32.0,25600.0,1,0
4230,0,32.0,25600.0,1,0
1378,0,36.0,26400.0,0,0
2723,0,36.0,26400.0,0,0
1002,1,34.0,26900.0,0,0
1140,1,34.0,26900.0,0,0


In [5]:
df = df.drop_duplicates()

**Мы загрузили и изучили данные, т.к. в данном проекте не требуется аналитика и предобработка, просто удалили обнаруженные дубликаты.**

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Разделим данные на признаки и таргет.

In [6]:
X = df.drop(labels = "Страховые выплаты",axis = 1)
y = df["Страховые выплаты"]
print(X.shape,y.shape)

(4847, 4) (4847,)


Добавим 1 вектор, что соответствует w0.

In [7]:
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
X.shape

(4847, 5)

Создадим различные варианты матриц для дальнейшего домножения на X.
Также посмотрим детерминант матриц , что будет подтверждать обратима или необратима матрица.

Обратимые матрицы:  

$X$ - исходная матрица;  
$E$ - единичная матрица;  
$R$ - рандомная(обратимая) матрица;  
$Q$ - матрица полученная $X.T @ X$  

Необратимая матрица:  
$M$ - матрица со всеми 0.


все матрицы имеют размерность равную n x n.  
где n = длина вектора признаков, т.е. 5

В задании указано, что нужно домножить признаки на обратимую матрицу,чтобы проверить изменится ли качество линейной регрессии.

In [8]:
def error_inv(X):
    try: 
        np.linalg.inv(X)
        print("Матрица обратима, детерминант = ",np.linalg.det(X))
    except np.linalg.LinAlgError:
        try:
            print("Матрица необратима, детерминант = ",np.linalg.det(X))
        except:
            print("Матрицa не удовлетваоряет требования к обратимой матрице, возможно неквадратная!")

In [9]:
error_inv(X)

Матрицa не удовлетваоряет требования к обратимой матрице, возможно неквадратная!


In [10]:
E = np.eye(5)
print(E)
error_inv(E)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
Матрица обратима, детерминант =  1.0


In [11]:
X_E = X.dot(E)
X_E.shape

(4847, 5)

In [12]:
np.random.seed(0)
R = []
for i in range(5):
    vector = np.random.rand(5)
    R.append(vector)
    
R = np.array(R)
print(R)
error_inv(R)

[[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]
 [0.64589411 0.43758721 0.891773   0.96366276 0.38344152]
 [0.79172504 0.52889492 0.56804456 0.92559664 0.07103606]
 [0.0871293  0.0202184  0.83261985 0.77815675 0.87001215]
 [0.97861834 0.79915856 0.46147936 0.78052918 0.11827443]]
Матрица обратима, детерминант =  0.009658225505885123


In [13]:
X_R = X.dot(R)
X_R.shape

(4847, 5)

In [14]:
Q = X.T @ X
print(Q)
error_inv(Q)

[[4.84700000e+03 2.41600000e+03 1.50373000e+05 1.93375000e+08
  5.83300000e+03]
 [2.41600000e+03 2.41600000e+03 7.49940000e+04 9.67618000e+07
  2.88800000e+03]
 [1.50373000e+05 7.49940000e+04 5.01429700e+06 5.99212070e+09
  1.80553000e+05]
 [1.93375000e+08 9.67618000e+07 5.99212070e+09 8.19683486e+12
  2.31029800e+08]
 [5.83300000e+03 2.88800000e+03 1.80553000e+05 2.31029800e+08
  1.28690000e+04]]
Матрица обратима, детерминант =  5.771461849364651e+27


In [15]:
X_Q = X.dot(Q)
X_Q.shape

(4847, 5)

In [16]:
M = np.copy(R)
M[:,:] = 0
print(M)
error_inv(M)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Матрица необратима, детерминант =  0.0


In [17]:
X_M = X.dot(M)
X_M.shape

(4847, 5)

Свойства обратной матрицы:
1. Матрица  А  называется обратной, если существует матрица  В , удовлетворяющая выражению:  𝐴𝐵=𝐵𝐴=𝐸 , где  𝐸  - единичная матрица.  А  и  В  называются взаимно обратными.  
2. Если матрица  А  обратима, то для нее существует только одна матрица обратная к  А .Обратная матрица обозначается как  𝐴−1 
3. Если матрица  А  обратима , то матрица  𝐴−1  также является обратимой
4. Произведение любого числа обратимых матриц дает обратимую матрицы
5. Определить необратимой матрицы  =0 , определитель обратимой матрицы  ≠0 
6.  (𝐴.𝑇)−1  =  (𝐴−1).𝑇

Предположим матрица  𝐴  имеет 2 обратных матрицы  𝐵  и  𝐶 , тогда:

𝐴𝐶=𝐸=𝐵𝐴⇒   
𝐵=𝐵𝐸=𝐵(𝐴𝐶)=(𝐵𝐴)𝐶=𝐸𝐶=𝐶⇒   
𝐵=𝐶 

Обозначения:

* 𝑋  — матрица признаков (нулевой столбец состоит из единиц)

* 𝑦  — вектор целевого признака

* 𝑃  — матрица, на которую умножаются признаки

* 𝑤  — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

* 𝑤∗  - вектор весов линейной регрессии матрицы полученной домножение исходной на обратимую матрицу(нулевой элемент равен сдвигу)

* 𝑦̂   - предсказания по исходной матрице признаков

* 𝑦̂ ∗  - предсказания по матрице полученной домноженной на обратимую матрицу

**Основные свойства обратной матрицы:**

- $$det(A) = \frac{1}  {det(A^{-1})} $$
- $$(AB)^{-1} = B^{-1}A^{-1}$$ для двух квадратных обратимых матриц $A$ и $B$
- $$(A^T)^{-1} = ((A)^{-1})^T$$

**Основные свойства единичной матрицы:**
- $$AE = EA = A$$
- $$AA^{-1}=E$$

**Сочетательное свойство матриц, ассоциативность:**
- $$A(BC) = (AB)C$$

**Исходная матрица признаков $X$**
  
Предсказания:

$$
\hat{y} = Xw
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Распишем формулу предсказания:
$$
\hat{y} = X (X^T X)^{-1} X^T y
$$

- по свойству единичной матрицы получаем:
$$
\hat{y} = E E y
$$

- или 
$$
\hat{y} = y
$$

Доказали формулу для вычисления предсказаний.

**Новая матрица признаков $X.dot(P)$**


Предсказания:

$$
\hat{y}* = XPw*
$$

Формула обучения:

$$
w* = ((XP)^T XP)^{-1} (XP)^T y
$$

Раскроем транспонирование матриц:
$$
w* = (P^T X^T XP)^{-1} P^TX^T y
$$

Еще раз раскроем скобки для произведения обратных матриц:
$$
w* = P^{-1}X^{-1}(X^T)^{-1}(P^T)^{-1}P^TX^T y
$$

Сокращаем произведение матриц $(P^T)^{-1}P^T$ по свойству единичной матрицы. Получаем $E$ или $1$, т.е. можно опустить ;
$$
w* = P^{-1}X^{-1}(X^T)^{-1}X^T y
$$
или
$$
w* = P^{-1}(X^TX)^{-1}X^T y
$$

Распишем предсказания по новым признакам с учетом полученной формулы для $w*$:


$$
\hat{y}* = XPP^{-1}(X^TX)^{-1}X^T y
$$

Сокращаем произведение матриц $P(P)^{-1}$ по свойству единичной матрицы. Получаем $E$ или $1$, т.е. можно опустить ;
$$
\hat{y}* = X(X^TX)^{-1}X^T y
$$

**Получили формулу для $\hat{y}*$ которая полностью совпадает с формулой для $\hat{y}$**

Из этого можно сделать вывод, что независимо от того, какую метрику будем использовать MSE,MAE или R2 **значения метрик не изменятся!**

Также из формул выше видно, что коэф.исходной и домноженной матриц связаны как: $w* = P^{-1}w$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Формула MSE согласно sklearn:
$${MSE}(y, \hat{y}) = \frac{1}{n_\text{samples}} \sum_{i=0}^{n_\text{samples} - 1} (y_i - \hat{y}_i)^2.$$

Формула MAE согласно sklearn:  
$$ {MAE}(y, \hat{y}) = \frac{1}{n_{\text{samples}}} \sum_{i=0}^{n_{\text{samples}}-1} \left| y_i - \hat{y}_i \right|.$$

***Примеры полученных метрик при перемножении исходной матрицы на обратимые и необратимую матрицы:***

In [18]:
def metrics(X,y,name):
    model = LinearRegression()
    model.fit(X,y)
    predict = model.predict(X)
    return print("Метрика MSE для матрицы",name,"равна",mean_squared_error(y,predict))

In [19]:
print(metrics(X,y,name = 'X'))
print(metrics(X_E,y,name = 'X_E'))
print(metrics(X_R,y,name = 'X_R'))
print(metrics(X_Q,y,name = 'X_Q'))
print(metrics(X_M,y,name = 'X_M'))

Метрика MSE для матрицы X равна 0.125272638266816
None
Метрика MSE для матрицы X_E равна 0.125272638266816
None
Метрика MSE для матрицы X_R равна 0.1252724079401175
None
Метрика MSE для матрицы X_Q равна 0.12527264044835446
None
Метрика MSE для матрицы X_M равна 0.2198540875868632
None


**По результатам видно, что метрика нулевой (необратимой матрицы) значительно хуже метрик для обратимых матриц и исходной матрицы.
Метрики обратимых матриц и исходной сопадают, не изменились UPD (в пределах погрешности для типа данных float)**

In [20]:
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

**Ответ:** 
* качество линейной регрессии не изменится

**Обоснование:**

* выше приведено доказательство того, что предсказания исходной матрицы = предсказаниям исходной матрицы домноженной на обратимую матрицу

## Алгоритм преобразования

**Алгоритм**

Основная задача проекта защитить данные клиентов страховой компании:
- разработать алгоритм преобразования данных, чтобы по ним было сложно восстановить персональную информаци;
- обосновать корректность его работы.

### Вариант решения №1:
- необходимо домножить исходную матрицу на обратимую. Как было доказано выше метрика MSE как и R2_score не изменится,т.к. результы предсказаний до умножения и после идентичны.  
**UPD** Такая матриц уже была сформирована ранее ($X\_R$ = $X@R$ , где $R$ - обратимая матрица и получена рандомно).

Т.к. матрица $X$_$R$ уже существует, то домножать повторно не имеет смысла, просто выведем значение матрицы $X\_R$ ниже:

In [21]:
display(X_R)
error_inv(R)
error_inv(X_R)

array([[ 4356.24731772,  1026.46913989, 41323.19018192, 38636.8133844 ,
        43156.44040217],
       [ 3348.86017226,   794.14261697, 31666.74842337, 29613.85939382,
        33064.27122128],
       [ 1853.22413334,   440.63948829, 17502.09281215, 16368.67895564,
        18272.73881367],
       ...,
       [ 2972.02381083,   698.29507812, 28238.6993774 , 26400.1317315 ,
        29495.49275038],
       [ 2870.67661372,   676.3275368 , 27242.04490423, 25469.93901557,
        28452.12196055],
       [ 3561.79119491,   837.62792897, 33822.2269927 , 31621.36986956,
        35325.40759919]])

Матрица обратима, детерминант =  0.009658225505885123
Матрицa не удовлетваоряет требования к обратимой матрице, возможно неквадратная!


### Вариант решения №2:
- необходимо применить метод StandardScaler.
Суть данного метода описывается формулой
$$z = \frac{(x - u)} {s}$$

$x$ - признак, который необходимо изменить  
$u$ - среднее значение столбца данного признака  
$s$ - стандратное отклонение(СКО) признака

Данный метод приведет все признаки в диапазон 0 $+-$ одно стандартное отклонение

In [22]:
# т.к. в некоторых ячейках numpy определяет значения как Nan, применим метод np.nan_to_num.(меняет на 0)
def ss(X):
    s = np.std(X,axis = 0)
    u = np.mean(X,axis = 0)
    X_new = []
    for i in range(X.shape[0]):
        vector = []
        for j in range(X.shape[1]):
            ss = (X[i,j] - u[j])/s[j]
            ss = np.nan_to_num(ss)
            vector.append(ss)
        X_new.append(vector)
    return X_new 

In [23]:
X_ss = np.array(ss(X))
X_ss

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


array([[ 0.        ,  1.0030995 ,  1.1754362 ,  0.97315092, -0.18517565],
       [ 0.        , -0.99691008,  1.76456423, -0.19011493, -0.18517565],
       [ 0.        , -0.99691008, -0.23847105, -1.89490109, -1.09546611],
       ...,
       [ 0.        , -0.99691008, -1.29890149, -0.60126924,  0.7251148 ],
       [ 0.        ,  1.0030995 , -1.06325028, -0.72160708,  1.63540526],
       [ 0.        ,  1.0030995 , -0.35629665,  0.07061707, -0.18517565]])

In [24]:
ss = StandardScaler()
X_ss = ss.fit_transform(X)

X_ss

array([[ 0.        ,  1.0030995 ,  1.1754362 ,  0.97315092, -0.18517565],
       [ 0.        , -0.99691008,  1.76456423, -0.19011493, -0.18517565],
       [ 0.        , -0.99691008, -0.23847105, -1.89490109, -1.09546611],
       ...,
       [ 0.        , -0.99691008, -1.29890149, -0.60126924,  0.7251148 ],
       [ 0.        ,  1.0030995 , -1.06325028, -0.72160708,  1.63540526],
       [ 0.        ,  1.0030995 , -0.35629665,  0.07061707, -0.18517565]])

## Проверка алгоритма

Cоздадим алгоритм для линейной регрессии.

In [25]:
X.dot(np.linalg.inv(X.T.dot(X))).dot(X.T).dot(y)

array([ 0.51932881,  0.69230097,  0.09412742, ..., -0.26164511,
       -0.19394186,  0.05044763])

In [26]:
model = LinearRegression()
model.fit(X,y)
predict = model.predict(X)
predict

array([ 0.51932881,  0.69230097,  0.09412742, ..., -0.26164511,
       -0.19394186,  0.05044763])

In [27]:
model = LinearRegression()
model.fit(X,y)
predict = model.predict(X)

print("R2_score для исходной матрицы = ",r2_score(y,predict))

R2_score для исходной матрицы =  0.4302010044852068


In [28]:
model = LinearRegression()
model.fit(X_R,y)
predict = model.predict(X_R)

print("R2_score для исходной матрицы домноженной на обратимую матрицу = ",r2_score(y,predict))

R2_score для исходной матрицы домноженной на обратимую матрицу =  0.43020205211957674


In [32]:
model = LinearRegression()
model.fit(X_ss,y)
predict = model.predict(X_ss)

print("R2_score для матрицы после применения  функции 'ss' (StandardScaler) = ",r2_score(y,predict))

R2_score для матрицы после применения  функции 'ss' (StandardScaler) =  0.4302010044852068


# Вывод

1. Загружены и изучены данные.
2. Качество линейной регресии не изменилось от использования исодной матрици и исходной матрицы, умноженную на обратимую.
3. Создан алгоритм преобразования данных.
4. Исследован алгоритм преобразования данных и проверена метрика R2 для данных без преобразования и с ним.
5. По результатам работы использования матричных операций можно увидеть, что алгоритм работает и данные очень просто зашифровать от распознования, имея правильную матрицу.